In [1]:
import os
import sys
PROJECT_PATH = os.path.dirname(os.path.dirname(os.path.realpath("__file__")))
sys.path.append(PROJECT_PATH)

import json
import pickle

import numpy as np
import pandas as pd
import torch
from torchsummary import summary

from src.data.downloader import Downloader
from src.model.baseline_model import BaselineModel
from src.model.linear_model import LinearModel
from src.model.mlp_model import MLPModel
from src.model.lstm_model import HydroForecast
from src.evaluation.evaluator import Evaluator

In [23]:
%%capture
gdrive_link = "https://drive.google.com/uc?export=download&id="
Downloader(gdrive_id="1XMqFFSc65UVE3EYh_tgPHtVRnonshjEO", file_name="all_analysis_for_paper.json") 
Downloader(files=json.load(open(os.path.join(PROJECT_PATH, "data", "all_analysis_for_paper.json"))))

# Data loading, preprocessing

In [8]:
data_training = pd.read_csv("../data/data_training.csv")
data_validation = pd.read_csv('../data/data_validation.csv')

data = pd.concat([data_training, data_validation])
data["Date"] = pd.to_datetime(data["Date"])
data = data.set_index('Date')
meta = pd.read_csv("../data/meta.csv")
data =  data.loc[data.index >= '2005']

data


,1515,1516,1518,1521,1719,1720,1722,1723,2040,2046,...,1732,1734,2049,2741,2742,2751,2545,744624,210888,210900
Date,,,,,,,,,,,,,,,,,,,,,
2005-01-01,-80.0,45.0,9.0,387.0,513.0,460.0,274.0,322.0,489.0,320,...,-47.0,35.0,142.0,86.0,229.0,244.0,222.0,82.696130,307.000000,288.000000
2005-01-02,-111.0,5.0,-36.0,362.0,505.0,462.0,243.0,310.0,482.0,299,...,-46.0,30.0,139.0,85.0,227.0,241.0,198.0,69.923412,309.000000,306.000000
2005-01-03,-123.0,-9.0,-67.0,341.0,499.0,465.0,214.0,270.0,466.0,274,...,-47.0,27.0,137.0,75.0,225.0,242.0,198.0,72.505006,304.000000,306.000000
2005-01-04,-132.0,-11.0,-72.0,336.0,498.0,464.0,203.0,260.0,464.0,226,...,-50.0,27.0,134.0,109.0,246.0,241.0,198.0,63.899692,293.000000,293.000000
2005-01-05,-127.0,-6.0,-76.0,331.0,496.0,466.0,193.0,258.0,458.0,210,...,-38.0,27.0,133.0,149.0,246.0,303.0,202.0,63.039160,287.000000,282.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-26,-27.0,75.0,-69.0,309.0,478.0,460.0,138.0,204.0,451.0,-78,...,-70.0,52.0,136.0,74.0,194.0,198.0,-18.0,31.000000,227.059349,212.846617
2020-12-27,-94.0,21.0,-50.0,333.0,488.0,460.0,206.0,252.0,464.0,20,...,-88.0,42.0,133.0,82.0,194.0,216.0,-17.0,19.000000,235.703288,221.319022
2020-12-28,-137.0,-20.0,-92.0,311.0,481.0,459.0,201.0,250.0,461.0,120,...,-86.0,40.0,133.0,83.0,197.0,215.0,-22.0,24.000000,256.980675,242.174172


In [9]:
s_data = data.copy()

In [10]:
norm_std = pd.read_csv('../data/params/paramsSTDS.csv',header=None)
norm_mean = pd.read_csv('../data/params/paramsMean.csv',header=None)

mean_szeged = pd.read_csv('../data/params/paramsYMean.csv')
std_szeged = pd.read_csv('../data/params/paramsYSTDS.csv')


#s_data.drop(s_data.tail(6).index,inplace=True)
s_data = (s_data - norm_mean.values.T).div(norm_std.values.T)
s_data

,1515,1516,1518,1521,1719,1720,1722,1723,2040,2046,...,1732,1734,2049,2741,2742,2751,2545,744624,210888,210900
Date,,,,,,,,,,,,,,,,,,,,,
2005-01-01,0.031084,0.154385,0.417111,0.610616,0.487570,0.326793,0.653387,0.735811,0.623071,0.651604,...,-0.698417,-0.614128,0.401449,-0.564918,-0.016621,-0.752997,1.537664,-0.474505,0.275750,-0.022900
2005-01-02,-0.171282,-0.076911,0.161921,0.415236,0.426870,0.342829,0.479478,0.669658,0.594934,0.575059,...,-0.680061,-0.731274,0.367912,-0.573618,-0.040140,-0.778571,1.318215,-0.610389,0.286429,0.100309
2005-01-03,-0.249618,-0.157865,-0.013877,0.251117,0.381345,0.366884,0.316789,0.449147,0.530621,0.483934,...,-0.698417,-0.801561,0.345554,-0.660612,-0.063659,-0.770046,1.318215,-0.582924,0.259732,0.100309
2005-01-04,-0.308369,-0.169430,-0.042231,0.212041,0.373758,0.358866,0.255079,0.394020,0.522582,0.308975,...,-0.753486,-0.801561,0.312016,-0.364830,0.183294,-0.778571,1.318215,-0.674473,0.200998,0.011325
2005-01-05,-0.275730,-0.140518,-0.064915,0.172965,0.358583,0.374902,0.198979,0.382994,0.498464,0.250655,...,-0.533209,-0.801561,0.300837,-0.016850,0.183294,-0.250047,1.354790,-0.683628,0.168962,-0.063970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-26,0.377066,0.327857,-0.025219,0.001030,0.222009,0.326793,-0.109570,0.085305,0.470327,-0.799102,...,-1.120614,-0.215833,0.334375,-0.669312,-0.428210,-1.145127,-0.656828,-1.024480,-0.151085,-0.537321
2020-12-27,-0.060307,0.015607,0.082528,0.188595,0.297883,0.326793,0.271909,0.349918,0.522582,-0.441893,...,-1.451029,-0.450124,0.300837,-0.599716,-0.428210,-0.991685,-0.647684,-1.152143,-0.104932,-0.479328
2020-12-28,-0.341009,-0.221471,-0.155649,0.016661,0.244771,0.318774,0.243859,0.338892,0.510523,-0.077394,...,-1.414316,-0.496983,0.300837,-0.591017,-0.392931,-1.000210,-0.693402,-1.098950,0.008677,-0.336576


In [11]:
s_data = s_data.iloc[0: -1 -6]
windowed_data = np.lib.stride_tricks.sliding_window_view(
    s_data, (15, 48)
).reshape((-1, 15, 48))

windowed_data_cnn = np.transpose(np.lib.stride_tricks.sliding_window_view(
    s_data, (15, 48)
).reshape((-1, 15, 48)),axes=(0,2,1))

win_data_torch = torch.tensor(windowed_data, dtype=torch.float32)

win_data_torch_cnn = torch.tensor(windowed_data_cnn, dtype=torch.float32)
win_data_torch.shape

torch.Size([5822, 15, 48])

In [12]:
y_true = pd.read_csv(os.path.join(PROJECT_PATH, "data", "y_true_2005-01-01_2020-12-24.csv"),
                     header=0, index_col=0)
y_true.index = pd.to_datetime(y_true.index)
y_true.drop(y_true.head(15).index,inplace=True)

In [13]:
y_true

,1day,2day,3day,4day,5day,6day,7day
2005-01-16,238.0,228.0,215.0,202.0,192.0,181.0,170.0
2005-01-17,228.0,215.0,202.0,192.0,181.0,170.0,163.0
2005-01-18,215.0,202.0,192.0,181.0,170.0,163.0,161.0
2005-01-19,202.0,192.0,181.0,170.0,163.0,161.0,156.0
2005-01-20,192.0,181.0,170.0,163.0,161.0,156.0,151.0
...,...,...,...,...,...,...,...
2020-12-20,117.0,115.0,112.0,112.0,111.0,107.0,110.0
2020-12-21,115.0,112.0,112.0,111.0,107.0,110.0,123.0
2020-12-22,112.0,112.0,111.0,107.0,110.0,123.0,155.0
2020-12-23,112.0,111.0,107.0,110.0,123.0,155.0,180.0


## Loading our lstm

In [109]:
lstm = torch.load('../data/lstm_model.pth')
lstm

HydroForecast(
  (lstm): LSTM(48, 12, num_layers=5, batch_first=True)
  (linear): Linear(in_features=12, out_features=7, bias=True)
)

## Load CNN

In [31]:
cnn = torch.load('../data/cnn_model.pth')
cnn

HydroForecast(
  (model): Sequential(
    (0): Conv1d(48, 5, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(5, 5, kernel_size=(5,), stride=(1,))
    (3): ReLU()
    (4): Conv1d(5, 7, kernel_size=(9,), stride=(1,))
  )
)

## Get predictions

In [32]:
def get_predictions(model, wd, st_id = "2275"):
    pred = pd.DataFrame(
        data=model(wd).detach().numpy(),
        index=y_true.index,
        columns=y_true.columns)

    return pred
    #return (pred + mean_szeged.iloc[0,0]) * std_szeged.iloc[0,0]

In [34]:
pred_lstm = get_predictions(model=lstm, wd = win_data_torch) * std_szeged.iloc[0,0] + mean_szeged.iloc[0,0]
pred_lstm.drop(pred_lstm.head(1).index,inplace=True)
pred_lstm.index = y_true.index.tolist()[0:-1]

pred_lstm


,1day,2day,3day,4day,5day,6day,7day
2005-01-16,237.218475,230.892242,222.210815,214.501266,211.315491,212.531555,218.088974
2005-01-17,214.809296,199.537888,186.197571,179.280502,180.448929,187.150635,197.257462
2005-01-18,208.240646,198.924164,188.705902,185.018951,188.999603,195.589691,206.675034
2005-01-19,192.461761,180.218506,168.993622,164.619553,168.151733,174.846207,186.167282
2005-01-20,180.693802,172.037964,163.428360,161.092438,165.828934,172.355362,183.546616
...,...,...,...,...,...,...,...
2020-12-19,117.023216,113.965782,110.781898,108.499237,107.526619,108.835838,110.496605
2020-12-20,114.820755,111.131889,109.138031,109.425301,111.166214,114.387962,117.375298
2020-12-21,113.346275,110.325478,109.524384,110.962044,113.535583,117.216187,120.581268
2020-12-22,109.478752,107.339523,108.768921,112.784172,117.553490,122.831802,127.140511


In [106]:
evaluator = Evaluator(y_true.drop(y_true.tail(1).index),pred_lstm)

In [108]:
evaluator.calculate_all_stats()

/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behav

,rmse,mae,r2,nse,wi,mape,rmae,mse,rrmse,rmsre,nnse,anse,nanse,fc,lm,nwi,corr
1day,16.147663,11.109172,0.990717,0.990717,0.990683,5.753010,3.333042,260.747035,7.193711,1.077783,0.990802,0.910761,0.918072,0.031344,-4.057264,0.990769,0.995365
2day,16.092100,11.262021,0.990781,0.990781,0.990743,5.936040,3.355893,258.955668,7.169655,1.074127,0.990866,0.909540,0.917044,0.031274,-4.126647,0.990828,0.995414
3day,17.592340,12.493724,0.988983,0.988983,0.988907,6.624424,3.534646,309.490437,7.838796,1.174320,0.989103,0.899656,0.908807,0.032492,-4.687097,0.989029,0.994529
4day,19.701839,13.945592,0.986184,0.986184,0.986016,7.369455,3.734380,388.162449,8.779452,1.315186,0.986372,0.888006,0.899285,0.034779,-5.347730,0.986209,0.993151
5day,23.605731,16.388239,0.980168,0.980168,0.979709,8.512484,4.048239,557.230532,10.519725,1.575836,0.980553,0.868399,0.883704,0.038464,-6.459347,0.980112,0.990181
6day,30.803500,20.849156,0.966230,0.966230,0.964779,10.480492,4.566088,948.855632,13.727748,2.056363,0.967333,0.832582,0.856591,0.043934,-8.489689,0.965977,0.983220
7day,39.756611,26.518922,0.943747,0.943747,0.939671,12.962744,5.149653,1580.588106,17.717764,2.654050,0.946743,0.787054,0.824439,0.049971,-11.070333,0.943103,0.971827


In [36]:
pred_lstm.to_csv('../data/metrics/lstm_model_raw_prediction_mod.csv')

In [39]:
pred_lstm

,1day,2day,3day,4day,5day,6day,7day
2005-01-16,237.218475,230.892242,222.210815,214.501266,211.315491,212.531555,218.088974
2005-01-17,214.809296,199.537888,186.197571,179.280502,180.448929,187.150635,197.257462
2005-01-18,208.240646,198.924164,188.705902,185.018951,188.999603,195.589691,206.675034
2005-01-19,192.461761,180.218506,168.993622,164.619553,168.151733,174.846207,186.167282
2005-01-20,180.693802,172.037964,163.428360,161.092438,165.828934,172.355362,183.546616
...,...,...,...,...,...,...,...
2020-12-19,117.023216,113.965782,110.781898,108.499237,107.526619,108.835838,110.496605
2020-12-20,114.820755,111.131889,109.138031,109.425301,111.166214,114.387962,117.375298
2020-12-21,113.346275,110.325478,109.524384,110.962044,113.535583,117.216187,120.581268
2020-12-22,109.478752,107.339523,108.768921,112.784172,117.553490,122.831802,127.140511


In [44]:
abs(y_true - pred_lstm).median()

1day     6.299103
2day     6.714249
3day     8.124420
4day     9.572449
5day    11.433746
6day    14.426758
7day    18.062500
dtype: float64

In [137]:
pred_cnn= pd.read_csv('../data/models/CNNmodel.csv')
pred_cnn.set_index('Date',inplace=True)
pred_cnn.index = pd.to_datetime(pred_cnn.index)
pred_cnn =pred_cnn * std_szeged.iloc[0, 0] + mean_szeged.iloc[0, 0]
pred_cnn.to_csv('../data/metrics/cnn_model_raw_prediction.csv')
evaluator_cnn = Evaluator(y_true.iloc[0:-1],pred_cnn)
evaluator_cnn.calculate_all_stats().to_csv('../data/metrics/cnn_metrics.csv')

/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behav

In [138]:
pred_lstm = pd.read_csv('../data/metrics/lstm_predictions_nn.csv')
pred_lstm.set_index('Date', inplace=True)
pred_lstm.index = pd.to_datetime(pred_lstm.index)
pred_lstm

,1day,2day,3day,4day,5day,6day,7day
Date,,,,,,,
2005-01-16,0.114853,0.072654,0.012716,-0.043672,-0.068863,-0.055186,-0.016495
2005-01-17,0.026456,-0.046063,-0.115417,-0.158980,-0.161714,-0.127381,-0.075018
2005-01-18,-0.013545,-0.078325,-0.136962,-0.167559,-0.159490,-0.119197,-0.065474
2005-01-19,-0.086040,-0.149822,-0.197626,-0.214159,-0.194762,-0.147823,-0.090421
2005-01-20,-0.128567,-0.175825,-0.208303,-0.213699,-0.188648,-0.140680,-0.084643
...,...,...,...,...,...,...,...
2020-12-19,-0.442136,-0.462325,-0.484613,-0.500706,-0.508159,-0.508559,-0.505779
2020-12-20,-0.459178,-0.482155,-0.501177,-0.511827,-0.514504,-0.511645,-0.506393
2020-12-21,-0.468882,-0.486630,-0.498511,-0.502621,-0.500624,-0.494943,-0.487949


In [139]:
pred_cnn_spatial = pd.read_csv('../data/models/CNNSpatmodel.csv')
pred_cnn_spatial .set_index('Date', inplace=True)
pred_cnn_spatial .index = pd.to_datetime(pred_cnn_spatial .index)
pred_cnn_spatial  = pred_cnn_spatial  * std_szeged.iloc[0, 0] + mean_szeged.iloc[0, 0]
#y_true.iloc[0:-1]
pred_cnn_spatial.to_csv('../data/metrics/cnn_spatial_model_raw_prediction.csv')
evaluator_cnn_spatial = Evaluator(y_true.iloc[0:-1], pred_cnn_spatial)
evaluator_cnn_spatial.calculate_all_stats().to_csv('../data/metrics/cnn_spatial_metrics.csv')


/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behav

In [140]:
pred_lstm_squared = pd.read_csv('../data/models/LSTMmodel_s.csv')
pred_lstm_squared.set_index('Date', inplace=True)
pred_lstm_squared.index = pd.to_datetime(pred_cnn_spatial.index)
pred_lstm_squared = pred_lstm_squared  * std_szeged.iloc[0, 0] + mean_szeged.iloc[0, 0]
#y_true.iloc[0:-1]
pred_lstm_squared.to_csv('../data/metrics/lstm_squared_raw_prediction.csv')
evaluator_cnn_spatial = Evaluator(y_true.iloc[0:-1], pred_lstm_squared)
evaluator_cnn_spatial.calculate_all_stats().to_csv('../data/metrics/lstm_squared_metrics.csv')


/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behav

In [2]:
def save_metrics(filename1,filename_to_save):
    pred = pd.read_csv('../data/models/'+ filename1)
    pred.set_index('Date',inplace=True)
    pred.index = pd.to_datetime(pred.index)
    pred = pred * std_szeged.iloc[0,0] + mean_szeged.iloc[0,0]
    pred.to_csv('../data/metrics/' + filename_to_save + '_raw_prediction.csv')
    evaluator = Evaluator(y_true.iloc[0:-1], pred)
    evaluator.calculate_all_stats().to_csv('../data/metrics/' + filename_to_save + '_metrics.csv')

In [155]:
save_metrics('CNNmodelloss3.csv','cnn_loss')

/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behav

In [156]:
save_metrics('CNNSpatmodelloss3.csv','cnn_loss_spatial')


/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behav

In [14]:
save_metrics('CNNSpatmodel.csv','cnn_spatial')

/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behav

In [15]:
save_metrics('LSTM_S_OPT.csv','lstm_s_opt')

/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behav

In [16]:
save_metrics('LSTMcub.csv','lstm_cub')

/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behav

In [17]:
save_metrics('LSTMexp.csv','lstm_exp')

/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/franzs/Schreibtisch/ecmi-hydrological-forecasting/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behav